In [ ]:
import numpy as np
class ReLU:
  def __init__(self):
    self.params,self.grads=[],[]
    self.out=None

  def forward(self,x):
    out=np.maximum(0,x)
    self.out=out
    return out

  def backward(self,dout):
      out=self.out
      dx=dout*np.where(out>0,1,0)
      return dx

In [ ]:
import numpy as np
class Sigmoid:
  def __init__(self):
    self.params,self.grads=[],[]
    self.out=None

  def forward(self,x):
    out=1/(1+np.exp(-x))
    self.out=out
    return out

  def backward(self,dout):
    out=self.out
    dx=dout*(out)*(1.0-out)
    return dx

In [ ]:
class MSELoss:
  def __init__(self):
    self.params=[]
    self.grads=[]

  def forward(self,x,t,actions):
    t=np.vstack((t,t)).T
    onehot_actions=np.zeros((len(actions),2))
    for i,action in enumerate(actions):
      onehot_actions[i][int(action)]=1

    self.x=x
    self.t=t
    self.actions=onehot_actions
    square=(x-t)*(x-t)*onehot_actions
    out=square.sum()/len(x)
    return out

  def backward(self,dout):
    return 2*dout*(self.x-self.t)*self.actions

In [ ]:
class Affine:
  def __init__(self,W,b):
    self.params=[W,b]
    self.grads=[np.zeros_like(W),np.zeros_like(b)]
    self.x=None

  def forward(self,x):
    W,b=self.params
    out=np.matmul(x,W)+b
    self.x=x
    return out

  def backward(self,dout):
    W,b=self.params
    dx=np.matmul(dout,W.T)
    dW=np.matmul(self.x.T,dout)
    db=np.sum(dout,axis=0)

    self.grads[0][...]=dW
    self.grads[1][...]=db

    return dx



In [ ]:
import numpy as np
class NeuralNet():
  def __init__(self,input_dim,output_dim,hidden_dim=(8,4)):

    W=[]
    b=[]
    W.append(np.random.rand(input_dim,hidden_dim[0]))
    b.append(np.random.rand(hidden_dim[0]))

    for i in range(len(hidden_dim)-1):
      W.append(np.random.rand(hidden_dim[i],hidden_dim[i+1]))
      b.append(np.random.rand(hidden_dim[i+1]))
    W.append(np.random.rand(hidden_dim[-1],output_dim))
    b.append(np.random.rand(output_dim))

    self.layers=[]
    for i in range(len(W)-1):
      self.layers.append(Affine(W[i],b[i]))
      self.layers.append(Sigmoid())
    self.layers.append(Affine(W[-1],b[-1]))
    self.loss_fn=MSELoss()

    self.params=[]
    self.grads=[]
    for layer in self.layers:
      self.params+=layer.params
      self.grads+=layer.grads
  def predict(self,x):
    self.batch_size=len(x)
    for layer in self.layers:
      x=layer.forward(x)
    return x
  def forward(self,x,t,actions):
    y=self.predict(x)
    loss=self.loss_fn.forward(y,t,actions)
    return loss
  def backward(self,dout=1):
    dout=self.loss_fn.backward(dout)
    for layer in reversed(self.layers):
      dout=layer.backward(dout)


# Optimizers

In [ ]:
class semi_GD():
  def __init__(self,model,learning_rate):
    self.lr=learning_rate
    self.model=model
  def update(self):
    for i in range(len(self.model.params)):
      self.model.params[i]-=self.lr*self.model.grads[i]

class Momentum():
  def __init__(self,model,lr,momentum):
    self.lr=lr
    self.momentum=momentum
    self.model=model
    self.v=[]
    for i in range(len(self.model.params)):
      self.v.append(np.zeros_like(self.model.grads[i]))
  def update(self):
    for i in range(len(self.model.params)):
      self.v[i]=self.momentum*self.v[i]+self.model.grads[i]
      self.model.params[i]-=self.lr*self.v

class RMSProp():
  def __init__(self,model,lr,gamma):
    self.model=model
    self.lr=lr
    self.gamma=gamma
    self.s=[]
    for i in range(len(self.model.params)):
      self.s.append(np.zeros_like(self.model.grads[i]))
  def update(self):
    epsilon=1e-6
    for i in range(len(self.model.params)):
      self.s[i]=self.gamma*self.s[i]+(1-self.gamma)*self.model.grads[i]*self.model.grads[i]
      self.model.params[i]-=self.lr/np.sqrt(self.s[i]+epsilon)*self.model.grads[i]



In [ ]:
import gym
env=gym.make('Blackjack-v1')
print(env.observation_space.sample())
env.reset()
s,r,terminated,truncated=env.step(1)
print(s,r,terminated,truncated)

(22, 5, 1)
(21, 1, False) 0.0 False {}


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Q-learning using neural network
  train function

In [ ]:
from tqdm import tqdm
def epsilon_greedy(Q,epsilon=0.01):
  p=np.random.rand()
  if p>epsilon:
    action=np.argmax(Q)
  else:
    action=np.random.choice(len(Q))
  return action

def train(env,strategy,model,optimizer,max_episode,max_epoch,batch_size,gamma):
  batch=[]
  counter=0
  for episode in tqdm(range(max_episode)):
    state,done=env.reset(),False
    state=np.array(list(state)).reshape(1,-1)

    while True:
      Qs=model.predict(state)[0]
      action=epsilon_greedy(Qs)
      next_state,reward,done,_=env.step(action)
      next_state=np.array(list(next_state)).reshape(1,-1)
      batch.append([state[0],action,reward,next_state[0],float(done)])

      #batch 데이터를 학습시킨다.
      #문제점:첫번째 학습에만 그래디언트가 업데이트되고 두번째학습부터는 그래디언트 업데이트가 되지 않는다.
      if len(batch)==batch_size:
        counter+=1
        states=np.array([x[0] for x in batch])
        actions=[x[1] for x in batch]
        rewards=np.array([x[2] for x in batch])
        next_states=np.array([x[3] for x in batch])
        dones=np.array([x[4] for x in batch])
        for epoch in range(max_epoch):
          Qns=model.predict(next_states)
          Qns_max=Qns.max(axis=1)
          Q_targets=rewards+gamma*Qns_max*(1-dones)
          model.forward(states,Q_targets,actions)
          model.backward()
          optimizer.update()
        if counter%100==0:
          print(f'{counter}th batch loss={model.forward(states,Q_targets,actions)}')
        batch=[]
      if done:
        break
      state=next_state


# Deep Q-learning using gradient descent

In [ ]:
import numpy as np
def learned_policy(model):
  states=[]
  for i in range(2):
    print('usable ace=',i)
    for j in range(12,22):
      print('Card sum=',j)
      for k in range(10):
        Qs=model.predict(np.array([j,k,i]).reshape(1,-1))
        print(f'best action for state:{[j,k,i]}:{Qs.argmax()}')
      print('\n')
    print('\n')




In [ ]:
max_episode=10**5
max_epoch=40
batch_size=128
decay_rate=1
lr=1e-4

model_GD=NeuralNet(3,2)
optimizer=semi_GD(model_GD,lr)

train(env,epsilon_greedy,model_GD,optimizer,max_episode,max_epoch,batch_size,decay_rate)
learned_policy(model_GD)

 13%|█▎        | 13222/100000 [00:03<00:23, 3655.72it/s]

100th batch loss=0.917914183712629


 26%|██▌       | 25849/100000 [00:07<00:18, 4079.27it/s]

200th batch loss=0.968452554257319


 39%|███▉      | 38825/100000 [00:10<00:15, 3922.59it/s]

...

600th batch loss=0.8905757560677321


 90%|████████▉ | 89879/100000 [00:23<00:02, 4071.17it/s]

700th batch loss=0.9391243946425842


100%|██████████| 100000/100000 [00:25<00:00, 3899.31it/s]

usable ace= 0
Card sum= 12
best action for state:[12, 0, 0]:0
best action for state:[12, 1, 0]:0
best action for state:[12, 2, 0]:0
best action for state:[12, 3, 0]:0
best action for state:[12, 4, 0]:0
best action for state:[12, 5, 0]:0
best action for state:[12, 6, 0]:0
best action for state:[12, 7, 0]:0
best action for state:[12, 8, 0]:0
best action for state:[12, 9, 0]:0


Card sum= 13
best action for state:[13, 0, 0]:0
best action for state:[13, 1, 0]:0
best action for state:[13, 2, 0]:0
best action for state:[13, 3, 0]:0
best action for state:[13, 4, 0]:0
best action for state:[13, 5, 0]:0
best action for state:[13, 6, 0]:0
best action for state:[13, 7, 0]:0
best action for state:[13, 8, 0]:0
best action for state:[13, 9, 0]:0


Card sum= 14
best action for state:[14, 0, 0]:0
best action for state:[14, 1, 0]:0
best action for state:[14, 2, 0]:0
best action for state:[14, 3, 0]:0
best action for state:[14, 4, 0]:0
best action for state:[14, 5, 0]:0
best action for state:[14, 6, 0]:

#deep Q-learning using RMSProp


In [ ]:
max_episode=10**7
max_epoch=40
batch_size=1024
decay_rate=1
lr=0.01
RMSProp_gamma=0.99

model_RMS=NeuralNet(3,2)
optimizer=RMSProp(model_RMS,lr,RMSProp_gamma)

train(env,epsilon_greedy,model_RMS,optimizer,max_episode,max_epoch,batch_size,decay_rate)
learned_policy(model_RMS)

  1%|          | 57733/10000000 [00:16<48:26, 3420.64it/s]

100th batch loss=2.510375910639123


  1%|▏         | 125549/10000000 [00:33<39:00, 4218.75it/s]

200th batch loss=4.776357246888992


  2%|▏         | 199341/10000000 [00:50<37:41, 4333.94it/s]

...

14700th batch loss=0.49290815297744073


 99%|█████████▉| 9936958/10000000 [42:13<00:17, 3698.93it/s]

14800th batch loss=0.45822251609291065


100%|██████████| 10000000/10000000 [42:30<00:00, 3921.11it/s]

usable ace= 0
Card sum= 12
best action for state:[12, 0, 0]:1
best action for state:[12, 1, 0]:1
best action for state:[12, 2, 0]:1
best action for state:[12, 3, 0]:1
best action for state:[12, 4, 0]:1
best action for state:[12, 5, 0]:1
best action for state:[12, 6, 0]:1
best action for state:[12, 7, 0]:1
best action for state:[12, 8, 0]:1
best action for state:[12, 9, 0]:1


Card sum= 13
best action for state:[13, 0, 0]:1
best action for state:[13, 1, 0]:1
best action for state:[13, 2, 0]:1
best action for state:[13, 3, 0]:1
best action for state:[13, 4, 0]:1
best action for state:[13, 5, 0]:1
best action for state:[13, 6, 0]:1
best action for state:[13, 7, 0]:1
best action for state:[13, 8, 0]:1
best action for state:[13, 9, 0]:1


Card sum= 14
best action for state:[14, 0, 0]:1
best action for state:[14, 1, 0]:1
best action for state:[14, 2, 0]:0
best action for state:[14, 3, 0]:0
best action for state:[14, 4, 0]:0
best action for state:[14, 5, 0]:0
best action for state:[14, 6, 0]:

In [ ]:
gpu_info=!nvidia-smi
gpu_info='\n'.join(gpu_info)
if gpu_info.find('failed')>=0:
  print('Not connected to GPU')
else:
  print(gpu_info)


Thu Feb 27 01:39:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----